In [ ]:
// run this cell to prevent Jupyter from displaying the null output cell
com.twosigma.beakerx.kernel.Kernel.showNullExecutionResult = false;

<a id='notebook_id'></a>
# Enumerations

What do the following groups have in common?:

* days of the week
* months of the year
* suits in a standard deck of playing cards
* ranks in a standard deck of playing cards
* Canadian coins
* Canadian provinces and territories
* planets of the solar system
* arithmetic operations

Each group has a fixed number of members and the number of members in each group never (or very rarely) change.

An *enumeration* (or *enumerated type*, *enum type*, or *enum*) is a special kind of type that represents a fixed set of constants. The fixed set of constants for some of the examples listed above are:

* Sunday, Monday, ..., Saturday
* January, February, ..., December
* clubs, diamonds, hearts, spades
* 2, 3, ..., ace
* nickel, dime, ..., toonie

Java provides a special-purpose kind of class for supporting enumerations.

Unfortunately, there seems to be a limitation in the beakerX kernels that prevents enumerations typed directly into a Jupyter cell from working correctly. This notebook imports pre-compiled enumeration classes where needed. 

## Old style enumerations: `int` enum pattern

Older Java code (before Java 1.5) and C code used *int* constants to represent enumerations. For example, the days of the week might be encoded like so:

```java
public static final int SUNDAY = 0;
public static final int MONDAY = 1;
public static final int TUESDAY = 2;
// and so on
```

and the months of the year might be encoded like so:

```java
public static final int JANUARY = 0;
public static final int FEBRUARY = 1;
public static final int MARCH = 2;
// and so on
```

See the Java source of the utility class [IntEnums.java](../resources/src/ca/queensu/cs/cisc124/notes/enums/IntEnums.java) for a complete example of how such enumerations could be defined in a Java program.

`int` enums can be found in the standard library; see for example the [Calendar](https://docs.oracle.com/en/java/javase/11/docs/api/java.base/java/util/Calendar.html) class.

Two major problems with using `int`s to represent enumerations is that there is no type safety and there is no expressive power. Any constructor or method with an `int`, `long`, `float`, or `double` parameter will accept a day or month:

In [1]:
%classpath add jar ../resources/jar/notes.jar

import ca.queensu.cs.cisc124.notes.enums.IntEnums;

// square root of Thursday?
double x = Math.sqrt(IntEnums.THURSDAY);
System.out.println("x is  : " + x);

x is  : 2.0


null

There is no way for a constructor or method to restrict a parameter to be only a day or only a month. This forces constructors and methods to have to validate their arguments:

In [2]:
%classpath add jar ../resources/jar/notes.jar

import ca.queensu.cs.cisc124.notes.enums.IntEnums;

public class DayUtils {

    /**
     * private ctor to prevent instantiation
     */
    private DayUtils() {
        throw new AssertionError();
    }
    
    public static String toString(int day) {
        if (day < IntEnums.SUNDAY || day > IntEnums.SATURDAY) {
            throw new IllegalArgumentException("day out of range");
        }
        switch (day) {
            case IntEnums.SUNDAY:
                return "SUNDAY";
            case IntEnums.MONDAY:
                return "MONDAY";
            case IntEnums.TUESDAY:
                return "TUESDAY";
            case IntEnums.WEDNESDAY:
                return "WEDNESDAY";
            case IntEnums.THURSDAY:
                return "THURSDAY";
            case IntEnums.FRIDAY:
                return "FRIDAY";
            case IntEnums.SATURDAY:
                return "SATURDAY";
        }
        return "";  // should never happen
    }
}

com.twosigma.beaker.javash.bkr7def7fef.DayUtils

Also observe that there is no easy way to translate the `int` value to a meaningful `String`; instead, we have to create a method such as `DayUtils.toString(int)` to perform the translation.

Even with argument validation, it is still possible to write silly code such as:

In [3]:
%classpath add jar ../resources/jar/notes.jar

import ca.queensu.cs.cisc124.notes.enums.IntEnums;

System.out.println(DayUtils.toString(IntEnums.JULY));  // July is SATURDAY?

SATURDAY


null

It is possible to perform arithmetic with days and months, and it is possible to compare days and months; for example:

In [4]:
%classpath add jar ../resources/jar/notes.jar

import ca.queensu.cs.cisc124.notes.enums.IntEnums;

// add Sunday and Monday?
int day = IntEnums.SUNDAY + IntEnums.MONDAY;
System.out.println("day is: " + day);

// TUESDAY equals MARCH?
boolean eq = IntEnums.TUESDAY == IntEnums.MARCH;
System.out.println("eq is : " + eq);

day is: 1
eq is : true


null

Another common old style enumeration is to use `String`s in place of `int`s. For example, the days of the week might be encoded like so:

```java
public static final String SUNDAY = "SUNDAY";
public static final String MONDAY = "MONDAY";
public static final String TUESDAY = "TUESDAY";
// and so on
```

and the months of the year might be encoded like so:

```java
public static final String JANUARY = "JANUARY";
public static final String FEBRUARY = "FEBRUARY";
public static final String MARCH = "MARCH";
// and so on
```

See the Java source of the utility class [StringEnums.java](../resources/src/ca/queensu/cs/cisc124/notes/enums/StringEnums.java) for a complete example of how such enumerations could be defined in a Java program.

Using `String`s to represent the enumerations provides human readable `String`s but there is still no type safety and no expressive power. Because the members of the enumeration are simply `String`s it becomes tempting to use `String` literals instead of the defined constants:

In [5]:
%classpath add jar ../resources/jar/notes.jar

import ca.queensu.cs.cisc124.notes.enums.StringEnums;


public class DayUtils {

    /**
     * private ctor to prevent instantiation
     */
    private DayUtils() {
        throw new AssertionError();
    }
    
    public static boolean isWeekend(String day) {
        if (day.equals(StringEnums.SUNDAY) || day.equals(StringEnums.SATURDAY)) {
            return true;
        }
        else if (day.equals(StringEnums.MONDAY) ||
                day.equals(StringEnums.TUESDAY) ||
                day.equals(StringEnums.WEDNESDAY) ||
                day.equals(StringEnums.THURSDAY) ||
                day.equals(StringEnums.FRIDAY)) {
            return false;
        }
        throw new IllegalArgumentException("bad day");
    }
}

com.twosigma.beaker.javash.bkr7def7fef.DayUtils

In [6]:
System.out.println("is Monday a weekend day? " + DayUtils.isWeekend("MONDAY"));

is Monday a weekend day? false


null

Using literals becomes problematic when the programmer uses an incorrect literal; the compiler is unable to catch errors such as the following:

In [7]:
System.out.println("is Monday a weekend day? " + DayUtils.isWeekend("Monday"));

ERROR:  java.lang.IllegalArgumentException

Also observe that any method that has a `String` parameter (such as `isWeekend(String)`) that is supposed to be a day (or month) needs to validate its argument to determine if the string corresponds to one of the enumeration strings.

## Java enumerations

Java enumerations are full fledged classes that implicitly inherit from the class `java.lang.Enum`. Because enumerations are classes they can have fields and methods, and can implement interfaces; however, enumerations have no non-private constructors so they cannot be extended. In its simplest form, the body of an enumeration consists simply of a list of the names of the constants of the enumeration:

```java
package ca.queensu.cs.cisc124.notes.enums;

/**
 * A simple enumeration of the days of the week.
 *
 */
public enum Day {
	SUNDAY, MONDAY, TUESDAY, WEDNESDAY, THURSDAY, FRIDAY, SATURDAY;
}
```

An enumeration is a type which means that the programmer can create variables of the type of the enumeration:

In [8]:
%classpath add jar ../resources/jar/notes.jar

import ca.queensu.cs.cisc124.notes.enums.Day;

Day d = Day.THURSDAY;
System.out.println(d);

THURSDAY


null

Notice that Java enumerations provide a compiler generated `toString` method that simply returns the name of the constant. The programmer can override the compiler generated `toString` method if desired.

### Number of instances of the enumeration

A Java enumeration exports exactly one instance for each enumeration member via a `public static final` field; for example, see the [documentation for the `Day` enumeration](../resources/doc/ca/queensu/cs/cisc124/notes/enums/Day.html#SUNDAY). Because an enumeration has no non-private constructors, the client cannot create new instances of the enumeration. This means that the only instances of the enumeration are the constants defined by the enumeration. For any program that uses the `Day` enumeration there is exactly one object representing the day `SUNDAY`, exactly one object representing the day `MONDAY`, and so on.

### Type safety

Enumerations are types that receive all of the benefits of compile-time type safety. Recall the method `isWeekend(String)` that returns `true` if the specified string corresponds to a weekend day; in that method we had to compare the string to all seven possible days to ensure that the string was a valid day (see the first method in the following cell). Now consider a similar method where the parameter is a `Day` enumeration (see the second method in the following cell):

In [9]:
%classpath add jar ../resources/jar/notes.jar

import ca.queensu.cs.cisc124.notes.enums.StringEnums;
import ca.queensu.cs.cisc124.notes.enums.Day;

public class DayUtils {

    /**
     * private ctor to prevent instantiation
     */
    private DayUtils() {
        throw new AssertionError();
    }
    
    public static boolean isWeekend(String day) {
        if (day.equals(StringEnums.SUNDAY) || day.equals(StringEnums.SATURDAY)) {
            return true;
        }
        else if (day.equals(StringEnums.MONDAY) ||
                day.equals(StringEnums.TUESDAY) ||
                day.equals(StringEnums.WEDNESDAY) ||
                day.equals(StringEnums.THURSDAY) ||
                day.equals(StringEnums.FRIDAY)) {
            return false;
        }
        throw new IllegalArgumentException("bad day");
    }
    
    public static boolean isWeekend(Day day) {
        return (day == Day.SUNDAY || day == Day.SATURDAY);
    }
}

com.twosigma.beaker.javash.bkre4497412.DayUtils

In `isWeekend(Day)` we never need to test if the argument `day` is a valid day because any `Day` instance is guaranteed to be a valid day.

### `compareTo`

Enumerations automatically implement the `Comparable` interface. The natural ordering is the order in which the constants of the enumeration are defined, and unfortunately, there is no way for the implementer of the enumeration to override this behavior.

In [9]:
%classpath add jar ../resources/jar/notes.jar

import ca.queensu.cs.cisc124.notes.enums.Day;

Day mon = Day.MONDAY;
Day thu = Day.THURSDAY;
Day sat = Day.SATURDAY;
System.out.println(mon.compareTo(thu));    // MONDAY is 3 days before THURSDAY
System.out.println(sat.compareTo(mon));    // SATURDAY is 5 days after MONDAY
System.out.println(thu.compareTo(thu));    // THURSDAY is THURSDAY

-3
5
0


null

Because enumerations implement `compareTo` users are able to sort collections of `Day`s:

In [10]:
%classpath add jar ../resources/jar/notes.jar

import java.util.Arrays;
import java.util.Collections;
import java.util.List;
import java.util.ArrayList;
import ca.queensu.cs.cisc124.notes.enums.Day;

List<Day> allTheDays = new ArrayList<>();
allTheDays.addAll(Arrays.asList(Day.values()));   // add all of the days to the list
Collections.shuffle(allTheDays);                  // shuffle the list
System.out.println(allTheDays);

allTheDays.sort(null);                            // sort the list
System.out.println(allTheDays);


[MONDAY, SATURDAY, THURSDAY, FRIDAY, WEDNESDAY, SUNDAY, TUESDAY]
[SUNDAY, MONDAY, TUESDAY, WEDNESDAY, THURSDAY, FRIDAY, SATURDAY]


null

### Fields, constructors, and methods

Enumerations can have fields and methods. Fields are useful if (constant) data is associated with each enumeration constant. For example, suppose that we wanted to create an enumeration of months and we wanted to be able to get the number of days in a particular month. Because February can have 28 or 29 days depending on whether or not the year is a leap year, the method that returns the number of days in a month requires a year parameter. A possible implementation of a `Month` enumeration is as follows:

```java
package ca.queensu.cs.cisc124.notes.enums;

/**
 * An enumeration for months of the Gregorian calendar. A month instance
 * can return the number of days in that month for a given year.
 */
public enum Month {
    /**
     * Each constant should have a Javadoc comment in production code but
     * doing so here takes up too much room.
     */
	JANUARY(31),
	FEBRUARY(28),
	MARCH(31), 
	APRIL(30),
	MAY(31),
	JUNE(30),
	JULY(31),
	AUGUST(31),
	SEPTEMBER(30),
	OCTOBER(31),
	NOVEMBER(30),
	DECEMBER(31);
	
	private final int days;
	
	/**
	 * Initializes this month to have the specified number of days.
	 * 
	 * @param days the number of days in this month
	 */
	private Month(int days) {
		this.days = days;
	}
	
	/**
	 * Returns the number of days in this month given the year.
	 * 
	 * @param year the year
	 * @return the number of days in this month
	 */
	public int days(int year) {
		if (this != Month.FEBRUARY) {
			return this.days;
		}
		if (year % 400 == 0 ||
			(year % 4 == 0 && year % 100 != 0)) {
			return this.days + 1;
		}
		return this.days;
	}
}


```



The `Month` enumeration has a `private final` field `days` that represents the number of days in the month. 

A `private` constructor is defined to initialize the field `days`. It is a compile time error if the access modifier of an enumeration constructor is present and not `private`. If there is no access modifier then the compiler sets the access modifier to `private` instead of package private.

The constructor is invoked where the enumeration constants are defined. The expression `JANUARY(31)` creates the enumeration instance `JANUARY` invoking the constructor with the value `31`.

There is no difference between methods in enumerations and methods in ordinary classes. In the `Month` enumeration the method `days(int)` returns the number of days in the month if the instance is not the month `FEBRUARY`. If the instance is the month `FEBRUARY` then the method tests if the argument `year` is a [leap year](https://en.wikipedia.org/wiki/Leap_year). If `year` is a leap year then the method returns `this.days + 1` to account for the extra day that occurs in February during leap years.

### The `values` method

Every enumeration has a compiler synthesized `public static` method named `values` that returns an array of all of the enumeration constants in the order that the constants were declared. For example:

In [11]:
%classpath add jar ../resources/jar/notes.jar

import java.util.Arrays;
import ca.queensu.cs.cisc124.notes.enums.Day;
import ca.queensu.cs.cisc124.notes.enums.Month;

Day[] allTheDays = Day.values();
System.out.println(Arrays.toString(allTheDays));

Month[] allTheMonths = Month.values();
System.out.println(Arrays.toString(allTheMonths));

[SUNDAY, MONDAY, TUESDAY, WEDNESDAY, THURSDAY, FRIDAY, SATURDAY]
[JANUARY, FEBRUARY, MARCH, APRIL, MAY, JUNE, JULY, AUGUST, SEPTEMBER, OCTOBER, NOVEMBER, DECEMBER]


null

The `values` method makes it easy to iterate over all of the enumeration constants:

In [19]:
%classpath add jar ../resources/jar/notes.jar

import ca.queensu.cs.cisc124.notes.enums.Month;

int year = 2020;
for (Month m : Month.values()) {
    int days = m.days(year);
    System.out.println(m + " has " + days + " days in the year " + year);
}

JANUARY has 31 days in the year 2020
FEBRUARY has 29 days in the year 2020
MARCH has 31 days in the year 2020
APRIL has 30 days in the year 2020
MAY has 31 days in the year 2020
JUNE has 30 days in the year 2020
JULY has 31 days in the year 2020
AUGUST has 31 days in the year 2020
SEPTEMBER has 30 days in the year 2020
OCTOBER has 31 days in the year 2020
NOVEMBER has 30 days in the year 2020
DECEMBER has 31 days in the year 2020


null

The year 2020 is a leap year; changing `year` to 2019 in the previous cell should produce a slightly different output.

### The `valueOf(String)` method

Enumerations make it easy to convert between strings and enumeration constants. Every enumeration has a compiler synthesized `public static` `valueOf(String)` method that returns the enumeration constant corresponding to the `String` equal to its name. For example, we can get the enumeration constant corresponding to the month `MAY` like so:

In [21]:
%classpath add jar ../resources/jar/notes.jar

import ca.queensu.cs.cisc124.notes.enums.Month;

Month m = Month.valueOf("MAY");
System.out.println(m + " has " + m.days(2020) + " days");

MAY has 31 days


null

## A better `Card` class

Up until now, the `Card` class has been implemented using the string enum pattern; there are a fixed number of ranks and suits all represented using a different string. A better implementation would use enumerations for the ranks, suits, and suit colours.

The `Color` enumeration looks like:

```java
/**
 * The two colours of cards in a standard 52-card French deck.
 * 
 */
public enum Colour {
    
    /**
     * The enum for black cards.
     */
    BLACK,
    
    
    /**
     * The enum for red cards.
     */
    RED
}
```

The `Rank` enumeration looks like:


```java
/**
 * The thirteen ranks of cards in a standard 52-card French deck. This
 * enumeration orders the ace as the highest rank;
 * the ranks in order from "smallest" rank to "greatest" rank is:
 * 
 * <ul>
 * <li><code>TWO</code>
 * <li><code>THREE</code>
 * <li><code>FOUR</code>
 * <li><code>FIVE</code>
 * <li><code>SIX</code>
 * <li><code>SEVEN</code>
 * <li><code>EIGHT</code>
 * <li><code>NINE</code>
 * <li><code>TEN</code>
 * <li><code>JACK</code>
 * <li><code>QUEEN</code>
 * <li><code>KING</code>
 * <li><code>ACE</code>
 * </ul>
 *
 */
public enum Rank {
	
	/**
	 * The rank 2.
	 */
	TWO,
	
	/**
	 * The rank 3.
	 */
	THREE,
	
	/**
	 * The rank 4.
	 */
	FOUR,
	
	/**
	 * The rank 5.
	 */
	FIVE,
	
	/**
	 * The rank 6.
	 */
	SIX,
	
	/**
	 * The rank 7.
	 */
	SEVEN,
	
	/**
	 * The rank 8.
	 */
	EIGHT,
	
	/**
	 * The rank 9.
	 */
	NINE,
	
	/**
	 * The rank 10.
	 */
	TEN,
	
	/**
	 * The rank jack.
	 */
	JACK,
	
	/**
	 * The rank queen.
	 */
	QUEEN,
	
	/**
	 * The rank king.
	 */
	KING,
	
	/**
	 * The rank ace.
	 */
	ACE
}
```


The `Suit` enumeration looks like:

```java
/**
 * The four suits of cards in a standard 52-card French deck. This
 * enumeration orders the suits in ascending alphabetical order;
 * the suits in order from "smallest" suit to "greatest" suit is:
 * 
 * <ul>
 * <li><code>CLUBS</code>
 * <li><code>DIAMONDS</code>
 * <li><code>HEARTS</code>
 * <li><code>SPADES</code>
 * </ul>
 *
 */
public enum Suit {
	/**
	 * The black suit clubs.
	 */
	CLUBS(Colour.BLACK),
	
	/**
	 *  The red suit diamonds.
	 */
	DIAMONDS(Colour.RED),
	
	/**
	 * The red suit hearts.
	 */
	HEARTS(Colour.RED),
	
	/**
	 * The black suit spades.
	 */
	SPADES(Colour.BLACK);
	
	private Colour color;
	
	private Suit(Colour c) {
		this.color = c;
	}
	
	/**
	 * Returns the color of this suit.
	 * 
	 * @return the color of this suit
	 */
	public Colour color() {
		return this.color;
	}
}
```

With theses three enumerations, the improved `Card` class can be implemented as shown in the following cell:

In [12]:
%classpath add jar ../resources/jar/notes.jar

package ca.queensu.cs.cisc124.notes.enums.cards;

/**
 * A class representing a playing card from a standard 52-card French deck.
 * 
 * <p>
 * This class compares cards using only the ranks of the cards.
 */
public class Card implements Comparable<Card> {
    private Rank rank;
    private Suit suit;

    /**
     * Initializes this card to have the specified rank and suit.
     *
     * @param rank the rank of this card
     * @param suit the suit of this card
     */
    public Card(Rank rank, Suit suit) {
        if (rank == null || suit == null) {
            throw new NullPointerException();
        }
        this.rank = rank;
        this.suit = suit;
    }

    /**
     * Returns the rank of this card. The rank is one of the strings in the array
     * {@code Card.RANKS}.
     * 
     * @return the rank of this card
     */
    public Rank rank() {
        return this.rank;
    }

    /**
     * Returns the suit of this card. The suit is one of the strings in the array
     * {@code Card.SUITS}.
     * 
     * @return the suit of this card
     */
    public Suit suit() {
        return this.suit;
    }

    /**
     * Returns the colour of this card.
     * 
     * @return the colour of this card
     */
    public Colour colour() {
        return this.suit.colour();
    }

    /**
     * Returns a string representation of this card. The returned string is the rank
     * of this card followed by the string <code>" of "</code> followed by the suit
     * of this card.
     */
    @Override
    public String toString() {
        return this.rank + " of " + this.suit;
    }

    /**
     * Compares this card to the specified object for equality. The result is true
     * if <code>obj</code> is a card having the same rank and the same suit as this
     * card, false otherwise.
     * 
     * @return true if obj is a card having the same rank and the same suit as this
     *         card, false otherwise
     */
    @Override
    public boolean equals(Object obj) {
        if (this == obj) {
            return true;
        }
        if (!(obj instanceof Card)) {
            return false;
        }
        Card other = (Card) obj;
        if (this.rank.equals(other.rank) && this.suit.equals(other.suit)) {
            return true;
        }
        return false;
    }

    /**
     * Returns a hash code for this card.
     * 
     * @return a hash code for this card
     */
    @Override
    public int hashCode() {
        int result = this.rank.hashCode();
        int c = this.suit.hashCode();
        result = 31 * result + c;
        return c;
    }

    /**
     * Compares this card with another card for order. Cards are compared using only
     * their ranks.
     * 
     * @param other a card to compare with this card
     * @return a negative value, zero, or a positive value if the rank of this card
     *         is less than, equal to, or greater than the rank of the other card
     */
    @Override
    public int compareTo(Card other) {
        return this.rank.compareTo(other.rank);
    }

}


ca.queensu.cs.cisc124.notes.enums.cards.Card

The notable changes in the `Card` class are:

- the arrays `RANKS` and `SUITS` are no longer required because the legal ranks and suits are represented as enumerations
- the constructor no longer needs to test if the rank and suit are strings equal to the legal ranks and suits because it is impossible for the caller to supply an invalid `Rank` or `Suit` instance
- the `colour` method delegates to the `Suit` enumeration to get the colour of the suit
- `toString` returns a slightly different string. Instead of returning strings such as `"2 of HEARTS"` the returned strings are of the form `"TWO of HEARTS"` where the rank is obtained using the `Rank` constant names
- `compareTo` is much easier to implement compared to the previous version

## Exercises

1. Create a list of all 52 possible playing cards. Use a pair of nested loops that iterate over the `values()` of the `Rank` and `Suit` enumerations.

2. If you have been attempting the exercises from previous notebooks examine the classes that you have created to determine if any of them should be enumerations and convert any such classes you find to enumerations.

3. Create an enumeration representing the eight planets of the solar system where each planet has a field representing its mass. Your enumeration should provide a public method that returns the mass of the planet.